In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset

pip install 'aif360[LawSchoolGPA]'


In [2]:
raw_df=pd.read_csv("data.csv",index_col=0)
raw_df.drop_duplicates()
display(raw_df)

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,51552.0,C++;Python;Git;PostgreSQL,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,46482.0,Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,77290.0,C;C++;Java;Perl;Ruby;Git;Ruby on Rails,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,46135.0,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL;Gi...,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,160932.0,C++;Python,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,<35,No,Undergraduate,1,Man,No,Dev,7,2,Germany,41058.0,C#;HTML/CSS;JavaScript;TypeScript;Docker;Kuber...,13,1
73458,>35,No,Undergraduate,1,Man,No,Dev,21,16,United States of America,115000.0,C#;HTML/CSS;Java;JavaScript;npm;ASP.NET Core ;...,11,1
73459,<35,No,Undergraduate,1,Man,No,Dev,4,3,Nigeria,57720.0,HTML/CSS;JavaScript;TypeScript;Docker;Express;...,12,1
73460,<35,Yes,Undergraduate,1,Man,Yes,Dev,5,1,United States of America,70000.0,C#;HTML/CSS;JavaScript;SQL;TypeScript;npm;Yarn...,15,1


In [ ]:
man_df=raw_df.copy().loc[raw_df["Gender"]=="Man"].loc[:,['Gender','Employed']]
display(man_df)
woman_df=raw_df.copy().loc[raw_df["Gender"]!="Man"].loc[:,['Gender','Employed']]
display(woman_df)

,Gender,Employed
0,Man,0
1,Man,1
2,Man,0
3,Man,0
4,Man,0
...,...,...
73456,Man,0
73457,Man,1
73458,Man,1
73459,Man,1


,Gender,Employed
58,Woman,0
61,Woman,0
95,Woman,0
114,NonBinary,0
125,Woman,1
...,...,...
73355,Woman,1
73430,Woman,1
73431,Woman,1
73453,Woman,1


In [4]:
print(man_df.shape[0]/raw_df.shape[0])
print(woman_df.shape[0]/raw_df.shape[0])

man_label=man_df["Employed"].value_counts()
print(man_label[1]/man_df.shape[0])
print(man_label[0]/man_df.shape[0])

women_label=woman_df["Employed"].value_counts()
print(women_label[1]/woman_df.shape[0])
print(women_label[0]/woman_df.shape[0])

0.9334485856633361
0.06655141433666385
0.5407667740947604
0.4592332259052397
0.4724892616076907
0.5275107383923092


In [ ]:
#Extract HaveWorkedWith column into several binary feature (Bag of Words Like)
computer_skill=pd.DataFrame()
feature_count=0
milestone=7346
startTime=time.time()
for index,row in raw_df.iterrows():
    hww_dict=dict()
    if(index==milestone):
        print(f"{milestone} entries have been examined,{round(time.time()-startTime)} seconds has passed since last report")
        startTime=time.time()
        milestone+=7346
    hww=row["HaveWorkedWith"]
    if hww is not np.nan:
        hww_list=str.split(hww,";")
        for ww in hww_list:
            column_name=ww
            if column_name not in computer_skill:
                computer_skill=pd.concat([computer_skill,pd.Series([np.nan]*index,name=column_name)],axis=1)
                feature_count+=1
            hww_dict[column_name]=1
        new_row=pd.DataFrame(hww_dict,index=[index])
    else:
        new_row=pd.Series([[np.nan]*feature_count],index=[index])
    computer_skill=pd.concat([computer_skill,new_row])
computer_skill=computer_skill.replace(np.nan,0)
display(computer_skill)
#Because It took too much time processing, I write it to local file so it can be later read from file
computer_skill.to_csv("computer_skill.csv")

In [ ]:
#get extrated individual computer skill read from file
computer_skill=pd.read_csv("computer_skill.csv",index_col=0)

In [3]:
def min_max_normalisation(series):
    return (series - series.min()) / (series.max() - series.min())

#Remove unworkable feature
rdf=raw_df.copy().drop("HaveWorkedWith",axis=1)
#normalize PreviousSalary feature
rdf["PreviousSalary"]=min_max_normalisation(rdf["PreviousSalary"])
#Create seperate dataframe to incorporate Computer Skill, However this dataset does not work. May come back later
#cs_rdf=pd.concat([rdf.drop(["Employed"],axis=1),computer_skill,rdf["Employed"]],axis=1)
display(rdf)
#display(cs_rdf)

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,0.230139,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,0.207505,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,0.345042,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,0.205956,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,0.718445,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,<35,No,Undergraduate,1,Man,No,Dev,7,2,Germany,0.183291,13,1
73458,>35,No,Undergraduate,1,Man,No,Dev,21,16,United States of America,0.513391,11,1
73459,<35,No,Undergraduate,1,Man,No,Dev,4,3,Nigeria,0.257675,12,1
73460,<35,Yes,Undergraduate,1,Man,Yes,Dev,5,1,United States of America,0.312497,15,1


In [4]:
folds_index=np.array_split(rdf.sample(frac=1).index,5)
raw_dataset=StandardDataset(df=rdf, label_name="Employed",
                                    favorable_classes=[1],
                                    scores_name="",
                                    protected_attribute_names=['Gender'],
                                    privileged_classes=[["Man"]],
                                    categorical_features=['Age','Accessibility','EdLevel','MentalHealth', 'MainBranch',"Country"])



In [8]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]
Accuracy_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateImpact_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateParity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualOpportunity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualizedOdds_data=pd.DataFrame(columns=Cross_Validation_Column)
#Read progress from local file if something happend that the jupyter kernel needs to be restarted (like fix bug from source library and reloaded it)
#merics_data=pd.read_csv("Benchmark_data.csv",index_col=0)

privileged_groups = [{"Gender":1}]
unprivileged_groups = [{"Gender":0}]

In [5]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]

Accuracy_data=pd.read_csv("Benchmark_job_accuracy.csv",index_col=0)
display(Accuracy_data)
DisparateImpact_data=pd.read_csv("Benchmark_job_disparate_impact.csv",index_col=0)
display(DisparateImpact_data)
DisparateParity_data=pd.read_csv("Benchmark_job_disparate_parity.csv",index_col=0)
display(DisparateParity_data)
EqualOpportunity_data=pd.read_csv("Benchmark_job_equal_opportunity.csv",index_col=0)
display(EqualOpportunity_data)
EqualizedOdds_data=pd.read_csv("Benchmark_job_equalized_odds.csv",index_col=0)
display(EqualizedOdds_data)

privileged_groups = [{"Gender":1}]
unprivileged_groups = [{"Gender":0}]

,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394
DisparateImpactRemover(0.5),0.779147,0.771660,0.780629,0.766812,0.770692,0.773788
DisparateImpactRemover(0.7),0.765194,0.758184,0.766472,0.767697,0.751089,0.761727
DisparateImpactRemover(1.0),0.758388,0.757912,0.765655,0.769875,0.741900,0.758746
CalibratedEqOddsPostprocessing,0.779623,0.773906,0.777158,0.770896,0.774571,0.775231
RejectOptionClassification(EO),0.775335,0.778942,0.773482,0.773346,0.773550,0.774931


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620
DisparateImpactRemover(0.5),0.946372,0.947166,0.961427,0.901990,0.937334,0.938858
DisparateImpactRemover(0.7),1.072553,1.068420,1.103823,0.888793,1.145665,1.055851
DisparateImpactRemover(1.0),1.125894,1.109566,1.120789,0.897858,1.224465,1.095715
CalibratedEqOddsPostprocessing,0.847485,0.858382,0.852012,0.887014,0.857272,0.860433
RejectOptionClassification(EO),0.829472,0.888091,0.845143,0.846720,0.846399,0.851165


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654
DisparateImpactRemover(0.5),-0.028681,-0.028274,-0.020610,-0.055572,-0.033769,-0.033381
DisparateImpactRemover(0.7),0.034168,0.032324,0.048501,-0.064902,0.064610,0.022940
DisparateImpactRemover(1.0),0.057738,0.050885,0.056100,-0.059128,0.095961,0.040311
CalibratedEqOddsPostprocessing,-0.085749,-0.079443,-0.084127,-0.063027,-0.080357,-0.078540
RejectOptionClassification(EO),-0.096324,-0.062873,-0.087434,-0.085965,-0.086642,-0.083848


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464
DisparateImpactRemover(0.5),-0.002052,-0.002939,0.042683,0.010315,0.029482,0.015498
DisparateImpactRemover(0.7),0.081578,0.066837,0.105363,-0.006440,0.136400,0.076748
DisparateImpactRemover(1.0),0.105623,0.082945,0.121816,0.009807,0.163987,0.096836
CalibratedEqOddsPostprocessing,-0.037051,-0.057054,-0.030618,0.004117,-0.016486,-0.027418
RejectOptionClassification(EO),-0.016806,-0.015723,-0.052287,-0.038530,-0.059720,-0.036613


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508
DisparateImpactRemover(0.5),0.018843,-0.010376,0.003681,-0.045663,0.000023,-0.006698
DisparateImpactRemover(0.7),0.060048,0.039606,0.075831,-0.048370,0.087180,0.042859
DisparateImpactRemover(1.0),0.081673,0.060656,0.075437,-0.051380,0.119183,0.057114
CalibratedEqOddsPostprocessing,-0.059355,-0.058534,-0.052965,-0.053012,-0.046501,-0.054074
RejectOptionClassification(EO),-0.092944,-0.025721,-0.041742,-0.058749,-0.059218,-0.055675


In [6]:
def cross_validation(row_name,train_test_process):
    global Accuracy_data
    global DisparateImpact_data
    global DisparateParity_data
    global EqualOpportunity_data
    global EqualizedOdds_data
    accuracy_row=[]
    disparateImpcat_row=[]
    disparateParity_row=[]
    EqualOpportunity_row=[]
    EqualizedOdss_row=[]
    for fold in range(5):
        test_fold=folds_index[fold]
        train_index=pd.Index([])
        for f in folds_index:
            if f is not test_fold:
                train_index=train_index.append(f)
        train_set=raw_dataset.subset(train_index)
        test_set=raw_dataset.subset(test_fold)
        test_pred=train_test_process(train_set,test_set)
        metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        #Record Result
        accuracy_row.append(metric.accuracy())
        disparateImpcat_row.append(metric.disparate_impact())
        disparateParity_row.append(metric.statistical_parity_difference())
        EqualOpportunity_row.append(metric.true_positive_rate_difference())
        EqualizedOdss_row.append(metric.false_positive_rate_difference())
    accuracy_row.append(np.average(accuracy_row))
    disparateImpcat_row.append(np.average(disparateImpcat_row))
    disparateParity_row.append(np.average(disparateParity_row))
    EqualOpportunity_row.append(np.average(EqualOpportunity_row))
    EqualizedOdss_row.append(np.average(EqualizedOdss_row))
    #Record Cross Validation
    Accuracy_data=pd.concat([Accuracy_data,pd.DataFrame([accuracy_row],index=[row_name],columns=Cross_Validation_Column)])
    Accuracy_data.columns.name="Accuracy"
    display(Accuracy_data)
    DisparateImpact_data=pd.concat([DisparateImpact_data,pd.DataFrame([disparateImpcat_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateImpact_data.columns.name="Disparate Impact"
    display(DisparateImpact_data)
    DisparateParity_data=pd.concat([DisparateParity_data,pd.DataFrame([disparateParity_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateParity_data.columns.name="Disparate Parity"
    display(DisparateParity_data)
    EqualOpportunity_data=pd.concat([EqualOpportunity_data,pd.DataFrame([EqualOpportunity_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualOpportunity_data.columns.name="Equal Opportuniy"
    display(EqualOpportunity_data)
    EqualizedOdds_data=pd.concat([EqualizedOdds_data,pd.DataFrame([EqualizedOdss_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualizedOdds_data.columns.name="Equalized Odds"
    display(EqualizedOdds_data)

In [39]:
#Train RandomForest as Baseline model on original Datasets
def baseline_CV(train_set, test_set):
    #Training
    baseline=RandomForestClassifier()
    baseline.fit(train_set.features,train_set.labels.ravel())
    #Testing
    pred=baseline.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Baseline",baseline_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.77287,0.77504


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682


In [43]:
#Zhange et al. implementation
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
def AdversarialDebiase_CV(train_set,test_set):
    #Training
    tf.disable_eager_execution()
    tf.reset_default_graph()
    with tf.Session() as sess:
        debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                unprivileged_groups = unprivileged_groups,
                                scope_name='debiased_classifier',
                                debias=True,
                                sess=sess)
        debiased_model.fit(train_set)
    #Testing
        test_pred=debiased_model.predict(test_set)
    return test_pred

cross_validation("AdversarialDebiase",AdversarialDebiase_CV)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 1.214884; batch adversarial loss: 0.718431
epoch 0; iter: 200; batch classifier loss: 0.503185; batch adversarial loss: 0.660491
epoch 0; iter: 400; batch classifier loss: 0.476218; batch adversarial loss: 0.521435
epoch 1; iter: 0; batch classifier loss: 0.462522; batch adversarial loss: 0.478203
epoch 1; iter: 200; batch classifier loss: 0.356507; batch adversarial loss: 0.427156
epoch 1; iter: 400; batch classifier loss: 0.364713; batch adversarial loss: 0.374287
epoch 2; iter: 0; batch classifier loss: 0.516833; batch adversarial loss: 0.365904
epoch 2; iter: 200; batch classifier loss: 0.500899; batch adversarial loss: 0.361319
epoch 2; iter: 400; batch classifier loss: 0.494977; batch adversarial loss: 0.372760
epoch 3; iter: 0; batch classifier loss: 0.649150; batch adversarial loss: 0.439147
epoch 3; iter: 200; batch classifier loss: 0.568104; batch adversarial loss: 0.400212
epoch 3; iter: 400; batch classifier loss: 0.518218; batch adv

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072


In [45]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation
from aif360.algorithms.inprocessing import GridSearchReduction
def GridSearchReduction_EO_CV(train_set,test_set):
    #Training
    gsr_eo_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="EqualizedOdds",prot_attr="Gender")
    gsr_eo_model.fit(train_set)
    #Testing
    test_pred=gsr_eo_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(EO)",GridSearchReduction_EO_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514


In [46]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation

def GridSearchReduction_DP_CV(train_set,test_set):
    #Training
    gsr_dp_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="DemographicParity",prot_attr="Gender")
    gsr_dp_model.fit(train_set)
    #Testing
    test_pred=gsr_dp_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(DP)",GridSearchReduction_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398


In [47]:
#Kamiran and Calders, 2012 implementation
from aif360.algorithms.preprocessing import Reweighing
def Reweight_CV(train_set,test_set):
    #Training
    reweight=Reweighing(privileged_groups=privileged_groups,unprivileged_groups=unprivileged_groups)
    reweight.fit(train_set)
    reweight_set=reweight.transform(train_set.copy(deepcopy=True))
    reweight_test = reweight.transform(test_set.copy(deepcopy=True))

    reweight_model=RandomForestClassifier()
    reweight_model.fit(reweight_set.features,reweight_set.labels.ravel(),sample_weight=train_set.instance_weights)
    #Testing
    pred=reweight_model.predict(reweight_test.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Reweight",Reweight_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508


In [49]:
#Feldman et al., 2015 implementation
from aif360.algorithms.preprocessing import DisparateImpactRemover

def DisparateImpactRemover_05_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.5,sensitive_attribute="Gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.5)",DisparateImpactRemover_05_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394
DisparateImpactRemover(0.5),0.779147,0.771660,0.780629,0.766812,0.770692,0.773788


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620
DisparateImpactRemover(0.5),0.946372,0.947166,0.961427,0.901990,0.937334,0.938858


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654
DisparateImpactRemover(0.5),-0.028681,-0.028274,-0.020610,-0.055572,-0.033769,-0.033381


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464
DisparateImpactRemover(0.5),-0.002052,-0.002939,0.042683,0.010315,0.029482,0.015498


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508
DisparateImpactRemover(0.5),0.018843,-0.010376,0.003681,-0.045663,0.000023,-0.006698


In [50]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_07_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.7,sensitive_attribute="Gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.7)",DisparateImpactRemover_07_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394
DisparateImpactRemover(0.5),0.779147,0.771660,0.780629,0.766812,0.770692,0.773788
DisparateImpactRemover(0.7),0.765194,0.758184,0.766472,0.767697,0.751089,0.761727


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620
DisparateImpactRemover(0.5),0.946372,0.947166,0.961427,0.901990,0.937334,0.938858
DisparateImpactRemover(0.7),1.072553,1.068420,1.103823,0.888793,1.145665,1.055851


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654
DisparateImpactRemover(0.5),-0.028681,-0.028274,-0.020610,-0.055572,-0.033769,-0.033381
DisparateImpactRemover(0.7),0.034168,0.032324,0.048501,-0.064902,0.064610,0.022940


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464
DisparateImpactRemover(0.5),-0.002052,-0.002939,0.042683,0.010315,0.029482,0.015498
DisparateImpactRemover(0.7),0.081578,0.066837,0.105363,-0.006440,0.136400,0.076748


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508
DisparateImpactRemover(0.5),0.018843,-0.010376,0.003681,-0.045663,0.000023,-0.006698
DisparateImpactRemover(0.7),0.060048,0.039606,0.075831,-0.048370,0.087180,0.042859


In [51]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_10_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=1.0,sensitive_attribute="Gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(1.0)",DisparateImpactRemover_10_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394
DisparateImpactRemover(0.5),0.779147,0.771660,0.780629,0.766812,0.770692,0.773788
DisparateImpactRemover(0.7),0.765194,0.758184,0.766472,0.767697,0.751089,0.761727
DisparateImpactRemover(1.0),0.758388,0.757912,0.765655,0.769875,0.741900,0.758746


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620
DisparateImpactRemover(0.5),0.946372,0.947166,0.961427,0.901990,0.937334,0.938858
DisparateImpactRemover(0.7),1.072553,1.068420,1.103823,0.888793,1.145665,1.055851
DisparateImpactRemover(1.0),1.125894,1.109566,1.120789,0.897858,1.224465,1.095715


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654
DisparateImpactRemover(0.5),-0.028681,-0.028274,-0.020610,-0.055572,-0.033769,-0.033381
DisparateImpactRemover(0.7),0.034168,0.032324,0.048501,-0.064902,0.064610,0.022940
DisparateImpactRemover(1.0),0.057738,0.050885,0.056100,-0.059128,0.095961,0.040311


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464
DisparateImpactRemover(0.5),-0.002052,-0.002939,0.042683,0.010315,0.029482,0.015498
DisparateImpactRemover(0.7),0.081578,0.066837,0.105363,-0.006440,0.136400,0.076748
DisparateImpactRemover(1.0),0.105623,0.082945,0.121816,0.009807,0.163987,0.096836


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508
DisparateImpactRemover(0.5),0.018843,-0.010376,0.003681,-0.045663,0.000023,-0.006698
DisparateImpactRemover(0.7),0.060048,0.039606,0.075831,-0.048370,0.087180,0.042859
DisparateImpactRemover(1.0),0.081673,0.060656,0.075437,-0.051380,0.119183,0.057114


In [52]:
#Pleiss et al., 2017 implementation
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

def CalibratedEqOddsPostprocessing_CV(train_set,test_set):
    #Training
    ceop_model=RandomForestClassifier()
    ceop_model.fit(train_set.features,train_set.labels.ravel())

    train_eop_set=train_set.copy(deepcopy=True)
    train_eop_set.scores=ceop_model.predict(train_set.features)

    ceop=CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
    ceop.fit(train_set,train_eop_set)
    #Testing
    pred=ceop_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=ceop.predict(test_pred)
    return test_pred

cross_validation("CalibratedEqOddsPostprocessing",CalibratedEqOddsPostprocessing_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394
DisparateImpactRemover(0.5),0.779147,0.771660,0.780629,0.766812,0.770692,0.773788
DisparateImpactRemover(0.7),0.765194,0.758184,0.766472,0.767697,0.751089,0.761727
DisparateImpactRemover(1.0),0.758388,0.757912,0.765655,0.769875,0.741900,0.758746
CalibratedEqOddsPostprocessing,0.779623,0.773906,0.777158,0.770896,0.774571,0.775231


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620
DisparateImpactRemover(0.5),0.946372,0.947166,0.961427,0.901990,0.937334,0.938858
DisparateImpactRemover(0.7),1.072553,1.068420,1.103823,0.888793,1.145665,1.055851
DisparateImpactRemover(1.0),1.125894,1.109566,1.120789,0.897858,1.224465,1.095715
CalibratedEqOddsPostprocessing,0.847485,0.858382,0.852012,0.887014,0.857272,0.860433


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654
DisparateImpactRemover(0.5),-0.028681,-0.028274,-0.020610,-0.055572,-0.033769,-0.033381
DisparateImpactRemover(0.7),0.034168,0.032324,0.048501,-0.064902,0.064610,0.022940
DisparateImpactRemover(1.0),0.057738,0.050885,0.056100,-0.059128,0.095961,0.040311
CalibratedEqOddsPostprocessing,-0.085749,-0.079443,-0.084127,-0.063027,-0.080357,-0.078540


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464
DisparateImpactRemover(0.5),-0.002052,-0.002939,0.042683,0.010315,0.029482,0.015498
DisparateImpactRemover(0.7),0.081578,0.066837,0.105363,-0.006440,0.136400,0.076748
DisparateImpactRemover(1.0),0.105623,0.082945,0.121816,0.009807,0.163987,0.096836
CalibratedEqOddsPostprocessing,-0.037051,-0.057054,-0.030618,0.004117,-0.016486,-0.027418


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508
DisparateImpactRemover(0.5),0.018843,-0.010376,0.003681,-0.045663,0.000023,-0.006698
DisparateImpactRemover(0.7),0.060048,0.039606,0.075831,-0.048370,0.087180,0.042859
DisparateImpactRemover(1.0),0.081673,0.060656,0.075437,-0.051380,0.119183,0.057114
CalibratedEqOddsPostprocessing,-0.059355,-0.058534,-0.052965,-0.053012,-0.046501,-0.054074


In [7]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_EO_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Equal opportunity difference")
    # with train_set.temporarily_ignore('labels', 'scores'):
    #     if train_set != train_roc_EO_set:
    #         display(train_set)
    #         display(train_roc_EO_set)
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(EO)",RejectOptionClassification_EO_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394
DisparateImpactRemover(0.5),0.779147,0.771660,0.780629,0.766812,0.770692,0.773788
DisparateImpactRemover(0.7),0.765194,0.758184,0.766472,0.767697,0.751089,0.761727
DisparateImpactRemover(1.0),0.758388,0.757912,0.765655,0.769875,0.741900,0.758746
CalibratedEqOddsPostprocessing,0.779623,0.773906,0.777158,0.770896,0.774571,0.775231
RejectOptionClassification(EO),0.775335,0.778942,0.773482,0.773346,0.773550,0.774931


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620
DisparateImpactRemover(0.5),0.946372,0.947166,0.961427,0.901990,0.937334,0.938858
DisparateImpactRemover(0.7),1.072553,1.068420,1.103823,0.888793,1.145665,1.055851
DisparateImpactRemover(1.0),1.125894,1.109566,1.120789,0.897858,1.224465,1.095715
CalibratedEqOddsPostprocessing,0.847485,0.858382,0.852012,0.887014,0.857272,0.860433
RejectOptionClassification(EO),0.829472,0.888091,0.845143,0.846720,0.846399,0.851165


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654
DisparateImpactRemover(0.5),-0.028681,-0.028274,-0.020610,-0.055572,-0.033769,-0.033381
DisparateImpactRemover(0.7),0.034168,0.032324,0.048501,-0.064902,0.064610,0.022940
DisparateImpactRemover(1.0),0.057738,0.050885,0.056100,-0.059128,0.095961,0.040311
CalibratedEqOddsPostprocessing,-0.085749,-0.079443,-0.084127,-0.063027,-0.080357,-0.078540
RejectOptionClassification(EO),-0.096324,-0.062873,-0.087434,-0.085965,-0.086642,-0.083848


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464
DisparateImpactRemover(0.5),-0.002052,-0.002939,0.042683,0.010315,0.029482,0.015498
DisparateImpactRemover(0.7),0.081578,0.066837,0.105363,-0.006440,0.136400,0.076748
DisparateImpactRemover(1.0),0.105623,0.082945,0.121816,0.009807,0.163987,0.096836
CalibratedEqOddsPostprocessing,-0.037051,-0.057054,-0.030618,0.004117,-0.016486,-0.027418
RejectOptionClassification(EO),-0.016806,-0.015723,-0.052287,-0.038530,-0.059720,-0.036613


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508
DisparateImpactRemover(0.5),0.018843,-0.010376,0.003681,-0.045663,0.000023,-0.006698
DisparateImpactRemover(0.7),0.060048,0.039606,0.075831,-0.048370,0.087180,0.042859
DisparateImpactRemover(1.0),0.081673,0.060656,0.075437,-0.051380,0.119183,0.057114
CalibratedEqOddsPostprocessing,-0.059355,-0.058534,-0.052965,-0.053012,-0.046501,-0.054074
RejectOptionClassification(EO),-0.092944,-0.025721,-0.041742,-0.058749,-0.059218,-0.055675


In [7]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_DP_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Statistical parity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(DP)",RejectOptionClassification_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\s

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.779487,0.775607,0.776409,0.770828,0.772870,0.775040
AdversarialDebiase,0.787722,0.783707,0.787912,0.782739,0.780493,0.784514
GridSearchReduction(EO),0.775403,0.776560,0.776137,0.771508,0.769330,0.773788
GridSearchReduction(DP),0.777717,0.775675,0.773074,0.771032,0.770623,0.773624
Reweight,0.778874,0.776220,0.777294,0.771372,0.773210,0.775394
DisparateImpactRemover(0.5),0.779147,0.771660,0.780629,0.766812,0.770692,0.773788
DisparateImpactRemover(0.7),0.765194,0.758184,0.766472,0.767697,0.751089,0.761727
DisparateImpactRemover(1.0),0.758388,0.757912,0.765655,0.769875,0.741900,0.758746
CalibratedEqOddsPostprocessing,0.779623,0.773906,0.777158,0.770896,0.774571,0.775231
RejectOptionClassification(EO),0.775335,0.778942,0.773482,0.773346,0.773550,0.774931


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.860933,0.874735,0.853075,0.886562,0.851269,0.865315
AdversarialDebiase,0.971719,1.010629,0.980306,0.901824,0.892739,0.951443
GridSearchReduction(EO),0.887817,0.884595,0.890223,0.886666,0.842620,0.878384
GridSearchReduction(DP),0.904155,0.895517,0.895866,0.887371,0.856458,0.887873
Reweight,0.857317,0.863490,0.856116,0.894830,0.846346,0.863620
DisparateImpactRemover(0.5),0.946372,0.947166,0.961427,0.901990,0.937334,0.938858
DisparateImpactRemover(0.7),1.072553,1.068420,1.103823,0.888793,1.145665,1.055851
DisparateImpactRemover(1.0),1.125894,1.109566,1.120789,0.897858,1.224465,1.095715
CalibratedEqOddsPostprocessing,0.847485,0.858382,0.852012,0.887014,0.857272,0.860433
RejectOptionClassification(EO),0.829472,0.888091,0.845143,0.846720,0.846399,0.851165


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.077792,-0.070014,-0.083587,-0.063444,-0.084149,-0.075797
AdversarialDebiase,-0.016546,0.006186,-0.010615,-0.054658,-0.065223,-0.028171
GridSearchReduction(EO),-0.063179,-0.064335,-0.062381,-0.063246,-0.088813,-0.068391
GridSearchReduction(DP),-0.053761,-0.058398,-0.058915,-0.062934,-0.080721,-0.062946
Reweight,-0.079982,-0.076458,-0.081731,-0.058399,-0.086699,-0.076654
DisparateImpactRemover(0.5),-0.028681,-0.028274,-0.020610,-0.055572,-0.033769,-0.033381
DisparateImpactRemover(0.7),0.034168,0.032324,0.048501,-0.064902,0.064610,0.022940
DisparateImpactRemover(1.0),0.057738,0.050885,0.056100,-0.059128,0.095961,0.040311
CalibratedEqOddsPostprocessing,-0.085749,-0.079443,-0.084127,-0.063027,-0.080357,-0.078540
RejectOptionClassification(EO),-0.096324,-0.062873,-0.087434,-0.085965,-0.086642,-0.083848


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.039187,-0.034692,-0.014631,0.012156,-0.024422,-0.020155
AdversarialDebiase,0.034786,0.044786,0.050288,0.014030,0.005875,0.029953
GridSearchReduction(EO),-0.023962,-0.036819,-0.019663,0.015044,-0.038195,-0.020719
GridSearchReduction(DP),-0.008738,-0.035506,-0.016624,0.021816,-0.023802,-0.012571
Reweight,-0.039588,-0.054155,-0.021502,0.015264,-0.037337,-0.027464
DisparateImpactRemover(0.5),-0.002052,-0.002939,0.042683,0.010315,0.029482,0.015498
DisparateImpactRemover(0.7),0.081578,0.066837,0.105363,-0.006440,0.136400,0.076748
DisparateImpactRemover(1.0),0.105623,0.082945,0.121816,0.009807,0.163987,0.096836
CalibratedEqOddsPostprocessing,-0.037051,-0.057054,-0.030618,0.004117,-0.016486,-0.027418
RejectOptionClassification(EO),-0.016806,-0.015723,-0.052287,-0.038530,-0.059720,-0.036613


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.041938,-0.061687,-0.066299,-0.061106,-0.047383,-0.055682
AdversarialDebiase,0.009067,0.012303,0.017810,-0.042871,-0.036671,-0.008072
GridSearchReduction(EO),-0.029037,-0.048067,-0.021943,-0.063132,-0.045393,-0.041514
GridSearchReduction(DP),-0.024382,-0.037685,-0.018929,-0.068791,-0.042202,-0.038398
Reweight,-0.045986,-0.055083,-0.056456,-0.054089,-0.040925,-0.050508
DisparateImpactRemover(0.5),0.018843,-0.010376,0.003681,-0.045663,0.000023,-0.006698
DisparateImpactRemover(0.7),0.060048,0.039606,0.075831,-0.048370,0.087180,0.042859
DisparateImpactRemover(1.0),0.081673,0.060656,0.075437,-0.051380,0.119183,0.057114
CalibratedEqOddsPostprocessing,-0.059355,-0.058534,-0.052965,-0.053012,-0.046501,-0.054074
RejectOptionClassification(EO),-0.092944,-0.025721,-0.041742,-0.058749,-0.059218,-0.055675


In [8]:
Accuracy_data.to_csv("Benchmark_job_accuracy.csv")
DisparateImpact_data.to_csv("Benchmark_job_disparate_impact.csv")
DisparateParity_data.to_csv("Benchmark_job_disparate_parity.csv")
EqualOpportunity_data.to_csv("Benchmark_job_equal_opportunity.csv")
EqualizedOdds_data.to_csv("Benchmark_job_equalized_odds.csv")
